In [1]:
import numpy as np
import pandas as pd
import re
import string

import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("dataset/newdf3.csv")

df.head()

,id,name,description,brand,Model
0,1,HUAWEI,"Screen Size\t:\t7.12"" (1080 x 2244) Camera\t:\...",HUAWEI,HUAWEI ENJOY MAX
1,2,Duranta,This bicycle has a strong steel frame. Its bra...,Duranta,Duranta Super Sports Bicycle
2,19,Cadillac,"Basic: 4 Years / 50000 Miles, Corrosion: 4 Yea...",Cadillac,Cadillac Escalade ESV
3,20,Aston Martin,"3 Years / Unlimited Miles Basic, 10 Years / Un...",Aston Martin,Aston Martin DB11 V8
4,21,Nike,you can buy the Gym Bag Traveling Bag NIKE Onl...,Nike,Gym bag


## EDA

In [3]:
# Check null values

df.isnull().sum()

id             0
name           0
description    0
brand          0
Model          0
dtype: int64

In [4]:
# Check duplicate value

df.duplicated().sum()

0

In [5]:
# Shape
df.shape

(153, 5)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           153 non-null    int64 
 1   name         153 non-null    object
 2   description  153 non-null    object
 3   brand        153 non-null    object
 4   Model        153 non-null    object
dtypes: int64(1), object(4)
memory usage: 6.1+ KB


In [7]:
type(df.loc[0, 'description'])

str

In [8]:
# Check the unique values in name column

unique_value ,num_count = np.unique(ar=df.name, return_counts=True)

In [9]:
len(unique_value)

151

In [10]:
num_count

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [11]:
df.name.value_counts()

ORAS Genuine Leather Wallet    2
Ladies Backpack                2
HUAWEI                         1
HANDWOVEN SILK SAREES          1
Ladies Purse                   1
                              ..
Haylou Smart Watch             1
Xiaomi Mi Band 6 CN            1
CURREN 8356 Watch              1
Curren                         1
Apple Magic Mouse              1
Name: name, Length: 151, dtype: int64

In [12]:
for i, j in zip(unique_value, num_count):
    print(f'Product Name: {i} -----> Total counts: {j}')

Product Name: 2pcs-bangles-100-101 -----> Total counts: 1
Product Name: AFGHANI OXIDISED SILVER JEWELLERY -----> Total counts: 1
Product Name: AVITA Essential 14 laptop -----> Total counts: 1
Product Name: All Swivel Chair -----> Total counts: 1
Product Name: Apple Magic Mouse -----> Total counts: 1
Product Name: Apple iMac PC -----> Total counts: 1
Product Name: Aston Martin -----> Total counts: 1
Product Name: Asus PC -----> Total counts: 1
Product Name: BMW -----> Total counts: 1
Product Name: Baby Bouncer For Playing Sleeping & Relxation -----> Total counts: 1
Product Name: Baby Kitchen Set -----> Total counts: 1
Product Name: Baby gym rack and game pad combo -----> Total counts: 1
Product Name: Black Panther -----> Total counts: 1
Product Name: Blender VIS-SBL-018 Rapid Power -----> Total counts: 1
Product Name: Brislet -----> Total counts: 1
Product Name: COTTON WOVEN SAREE -----> Total counts: 1
Product Name: CPU Liquid Cooler -----> Total counts: 1
Product Name: CURREN 8356 Wat

## Text preprocessing

In [13]:
df.head(5)

,id,name,description,brand,Model
0,1,HUAWEI,"Screen Size\t:\t7.12"" (1080 x 2244) Camera\t:\...",HUAWEI,HUAWEI ENJOY MAX
1,2,Duranta,This bicycle has a strong steel frame. Its bra...,Duranta,Duranta Super Sports Bicycle
2,19,Cadillac,"Basic: 4 Years / 50000 Miles, Corrosion: 4 Yea...",Cadillac,Cadillac Escalade ESV
3,20,Aston Martin,"3 Years / Unlimited Miles Basic, 10 Years / Un...",Aston Martin,Aston Martin DB11 V8
4,21,Nike,you can buy the Gym Bag Traveling Bag NIKE Onl...,Nike,Gym bag


In [14]:
df.loc[0, 'description']

'Screen Size\t:\t7.12" (1080 x 2244) Camera\t:\t16 + 2 | 8 MP RAM\t:\t4GB Battery\t:\t5000 mAh Operating system\t:\tAndroid Soc\t:\tQualcomm SDM636 Snapdragon 636 Processor\t:\tOcta'

In [15]:
# Make lower case the text

def makeLowerCase(text):
    return text.lower()

In [16]:
df['name'] = df['name'].apply(func=makeLowerCase)
df['description'] = df['description'].apply(func=makeLowerCase)
df['brand'] = df['brand'].apply(func=makeLowerCase)
df['Model'] = df['Model'].apply(func=makeLowerCase)

In [17]:
df.head()

,id,name,description,brand,Model
0,1,huawei,"screen size\t:\t7.12"" (1080 x 2244) camera\t:\...",huawei,huawei enjoy max
1,2,duranta,this bicycle has a strong steel frame. its bra...,duranta,duranta super sports bicycle
2,19,cadillac,"basic: 4 years / 50000 miles, corrosion: 4 yea...",cadillac,cadillac escalade esv
3,20,aston martin,"3 years / unlimited miles basic, 10 years / un...",aston martin,aston martin db11 v8
4,21,nike,you can buy the gym bag traveling bag nike onl...,nike,gym bag


In [18]:
def simplePreprocessing(text):
    patt = re.compile(pattern=r"[\t:\d\.]")
    a = patt.sub(repl="", string=text)
    b = a.translate(str.maketrans('','',string.punctuation))
    c = " ".join(b.split())
    return c

In [19]:
df['description'] = df['description'].apply(func=simplePreprocessing)

df.head()

,id,name,description,brand,Model
0,1,huawei,screen size x camera mp ramgb battery mah oper...,huawei,huawei enjoy max
1,2,duranta,this bicycle has a strong steel frame its brak...,duranta,duranta super sports bicycle
2,19,cadillac,basic years miles corrosion years miles drivet...,cadillac,cadillac escalade esv
3,20,aston martin,years unlimited miles basic years unlimited mi...,aston martin,aston martin db11 v8
4,21,nike,you can buy the gym bag traveling bag nike onl...,nike,gym bag


In [20]:
patt = re.compile(pattern=r" (mm|mm x|w mm|mp|x|w|mp) ")
patt2 = re.compile(pattern=r" (ww|x x|f|w|mm|n m|a l v|n) ")
patt3 = re.compile(pattern=r" (q|v|l|hz – khz) ")

mypatt = [patt, patt2, patt3]

def removeSingleAlphabet(text):
    for i in mypatt:
        text = i.sub(repl=" ", string=text)
    return text

In [21]:
df['description'] = df['description'].apply(func=removeSingleAlphabet)

df.head()

,id,name,description,brand,Model
0,1,huawei,screen size camera ramgb battery mah operating...,huawei,huawei enjoy max
1,2,duranta,this bicycle has a strong steel frame its brak...,duranta,duranta super sports bicycle
2,19,cadillac,basic years miles corrosion years miles drivet...,cadillac,cadillac escalade esv
3,20,aston martin,years unlimited miles basic years unlimited mi...,aston martin,aston martin db11 v8
4,21,nike,you can buy the gym bag traveling bag nike onl...,nike,gym bag


In [22]:
df['tags'] = df['description'] + df['brand'] + df['Model']

df.head()

,id,name,description,brand,Model,tags
0,1,huawei,screen size camera ramgb battery mah operating...,huawei,huawei enjoy max,screen size camera ramgb battery mah operating...
1,2,duranta,this bicycle has a strong steel frame its brak...,duranta,duranta super sports bicycle,this bicycle has a strong steel frame its brak...
2,19,cadillac,basic years miles corrosion years miles drivet...,cadillac,cadillac escalade esv,basic years miles corrosion years miles drivet...
3,20,aston martin,years unlimited miles basic years unlimited mi...,aston martin,aston martin db11 v8,years unlimited miles basic years unlimited mi...
4,21,nike,you can buy the gym bag traveling bag nike onl...,nike,gym bag,you can buy the gym bag traveling bag nike onl...


In [23]:
df2 = df.drop(columns=['description', 'brand', 'Model'])

df2.head()

,id,name,tags
0,1,huawei,screen size camera ramgb battery mah operating...
1,2,duranta,this bicycle has a strong steel frame its brak...
2,19,cadillac,basic years miles corrosion years miles drivet...
3,20,aston martin,years unlimited miles basic years unlimited mi...
4,21,nike,you can buy the gym bag traveling bag nike onl...


In [24]:
df2.shape

(153, 3)

In [25]:
df2.isnull().sum().sum()

0

In [26]:
df2.loc[1, 'tags']

'this bicycle has a strong steel frame its brake system is very strong which allows you to maintain control in all situations its seat is adjustable so you can change it as needed its front and rear wheels are fitted with mudguards so that the rider does not get mud while riding on the roaddurantaduranta super sports bicycle'

In [27]:
print(df2.loc[1, 'tags'].split())

['this', 'bicycle', 'has', 'a', 'strong', 'steel', 'frame', 'its', 'brake', 'system', 'is', 'very', 'strong', 'which', 'allows', 'you', 'to', 'maintain', 'control', 'in', 'all', 'situations', 'its', 'seat', 'is', 'adjustable', 'so', 'you', 'can', 'change', 'it', 'as', 'needed', 'its', 'front', 'and', 'rear', 'wheels', 'are', 'fitted', 'with', 'mudguards', 'so', 'that', 'the', 'rider', 'does', 'not', 'get', 'mud', 'while', 'riding', 'on', 'the', 'roaddurantaduranta', 'super', 'sports', 'bicycle']


In [28]:
print(word_tokenize(text=df2.loc[1, 'tags']))

['this', 'bicycle', 'has', 'a', 'strong', 'steel', 'frame', 'its', 'brake', 'system', 'is', 'very', 'strong', 'which', 'allows', 'you', 'to', 'maintain', 'control', 'in', 'all', 'situations', 'its', 'seat', 'is', 'adjustable', 'so', 'you', 'can', 'change', 'it', 'as', 'needed', 'its', 'front', 'and', 'rear', 'wheels', 'are', 'fitted', 'with', 'mudguards', 'so', 'that', 'the', 'rider', 'does', 'not', 'get', 'mud', 'while', 'riding', 'on', 'the', 'roaddurantaduranta', 'super', 'sports', 'bicycle']


In [29]:
df3 = df2.copy()

## Create WordNetLemmatizer object

In [30]:
lemmatizer = WordNetLemmatizer()

## Remove stopwords and lemmatize those words

In [31]:
def textLemmatizer(text):
    lis = []
    words = word_tokenize(text=text)
    words = [lemmatizer.lemmatize(word=word) for word in words if word not in stopwords.words('english')]
    text = ' '.join(words)
    return text

In [32]:
text = """this bicycle has a strong steel frame its brake system is very strong which allows you to maintain control in all situations its seat is adjustable so you can change it as needed its front and rear wheels are fitted with mudguards so that the rider does not get mud while riding on the roaddurantaduranta super sports bicycle"""

result = textLemmatizer(text)

result

'bicycle strong steel frame brake system strong allows maintain control situation seat adjustable change needed front rear wheel fitted mudguard rider get mud riding roaddurantaduranta super sport bicycle'

In [33]:
df2['tags'] = df2['tags'].apply(textLemmatizer)

df2.head()

,id,name,tags
0,1,huawei,screen size camera ramgb battery mah operating...
1,2,duranta,bicycle strong steel frame brake system strong...
2,19,cadillac,basic year mile corrosion year mile drivetrain...
3,20,aston martin,year unlimited mile basic year unlimited mile ...
4,21,nike,buy gym bag traveling bag nike online banglade...


In [34]:
df2.loc[1, 'tags']

'bicycle strong steel frame brake system strong allows maintain control situation seat adjustable change needed front rear wheel fitted mudguard rider get mud riding roaddurantaduranta super sport bicycle'

## Apply TF-IDF 

In [35]:
tf_idf = TfidfVectorizer(max_features=2000)

In [36]:
tf_idf_vectors = tf_idf.fit_transform(raw_documents=df2['tags']).toarray()

In [37]:
tf_idf.get_feature_names_out()

array(['010', '013', '018', ..., 'zari', 'zigzag', 'zoom'], dtype=object)

In [38]:
tf_idf_vectors[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [39]:
tf_idf_vectors.shape

(153, 1290)

In [40]:
tf_idf_vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Check the similarity between the words

In [41]:
similarity = cosine_similarity(X=tf_idf_vectors)

In [42]:
similarity.shape

(153, 153)

In [43]:
similarity[0]

array([1.        , 0.        , 0.        , 0.        , 0.        ,
       0.05401968, 0.        , 0.        , 0.        , 0.        ,
       0.03950974, 0.        , 0.        , 0.        , 0.        ,
       0.07530558, 0.10610892, 0.        , 0.07952949, 0.106362  ,
       0.        , 0.18138298, 0.        , 0.        , 0.        ,
       0.        , 0.05370973, 0.        , 0.05889805, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.06830434, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.04440784, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [44]:
similarity[1]

array([0.        , 1.        , 0.        , 0.        , 0.03876492,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.22760906, 0.47023878, 0.1370756 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.04521035, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.02356674, 0.03089456,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.02051336, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.02511157, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.02218058,
       0.        , 0.06775046, 0.        , 0.        , 0.        ,
       0.09242149, 0.        , 0.        , 0.        , 0.     

In [45]:
similarity[1].shape

(153,)

### Demo test start

In [46]:
df2[df2['name'] == 'huawei']

,id,name,tags
0,1,huawei,screen size camera ramgb battery mah operating...


In [47]:
df2[df2['name'] == 'huawei'].index[0]

0

In [48]:
list(enumerate(similarity[df2[df2['name'] == 'huawei'].index[0]]))[:20]

[(0, 1.0000000000000002),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0540196789475405),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.039509739272212636),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.07530557971545761),
 (16, 0.10610891719642623),
 (17, 0.0),
 (18, 0.07952949370185838),
 (19, 0.1063619965093359)]

In [49]:
sorted(list(enumerate(similarity[df2[df2['name'] == 'huawei'].index[0]])), reverse=True, key=lambda x:x[1])[:20]

[(0, 1.0000000000000002),
 (21, 0.1813829798712177),
 (19, 0.1063619965093359),
 (16, 0.10610891719642623),
 (18, 0.07952949370185838),
 (15, 0.07530557971545761),
 (43, 0.06830434260897499),
 (119, 0.059969357622131986),
 (147, 0.058989077963006266),
 (28, 0.0588980543843816),
 (5, 0.0540196789475405),
 (26, 0.05370972797155801),
 (152, 0.04977419626220306),
 (58, 0.04440784421909613),
 (115, 0.04225548089151964),
 (10, 0.039509739272212636),
 (76, 0.03659645884976964),
 (90, 0.03653111949121198),
 (139, 0.0364418945138332),
 (78, 0.023636713535303392)]

In [50]:
sorted(list(enumerate(similarity[df2[df2['name'] == 'huawei'].index[0]])), reverse=True, key=lambda x : x[1])[:10]

[(0, 1.0000000000000002),
 (21, 0.1813829798712177),
 (19, 0.1063619965093359),
 (16, 0.10610891719642623),
 (18, 0.07952949370185838),
 (15, 0.07530557971545761),
 (43, 0.06830434260897499),
 (119, 0.059969357622131986),
 (147, 0.058989077963006266),
 (28, 0.0588980543843816)]

In [51]:
all_distance = sorted(list(enumerate(similarity[df2[df2['name'] == 'huawei'].index[0]])), reverse=True, key=lambda x : x[1])[:10]

In [52]:
for i in all_distance:
    print(df2.loc[i[0], 'name'])

huawei
realme
redmi
motorola moto
datsun
haier
apple imac pc
dell inspiron 15 3515 laptop
diamond storm  pro rgb gaming casing
oppo


### Demo Test End

### Recommendar Function

In [74]:
def productRecommender(productName):
    productName = productName.lower()
    product_index = df2[df2['name'] == productName].index[0]
    distances = sorted(list(enumerate(similarity[product_index])), reverse=True, key=lambda x : x[1])
    for i in distances[1:16]:
        print(df2.loc[i[0], 'name'])

In [75]:
productRecommender('Nike')

ladies backpack
laptop pouch bag
ladies backpack
baby gym rack and game pad combo
lg 32lk510b 32" hd led television
duranta
led light
huawei
cadillac
aston martin
lakmé
lakmé 9 to 5
hijab
forever bicycle
express


In [55]:
productRecommender('oneplus')

forever bicycle
nokia
oppo
iphone
redmi
chuwi herobook  laptop
diamond storm  pro rgb gaming casing
huawei
duranta
cadillac
aston martin
nike
ladies backpack
ladies backpack
lakmé


In [56]:
productRecommender('duranta')

express
forever bicycle
super 16
diamond storm  pro rgb gaming casing
playtime king slider
semi fitted panjabi
kick scooter kids
x corolla
zoom hd digital camera
nike
dell inspiron 15 3515 laptop
gfc monet ceiling fan
electronic dry iron
blender vis-sbl-018 rapid power
michael kors


## Apply porterStemmer

In [57]:
ps = PorterStemmer()

### Remove stopwords and stem those words

In [58]:
def textPorterStemmer(text):
    lis = []
    words = word_tokenize(text=text)
    words = [ps.stem(word=word) for word in words if word not in stopwords.words('english')]
    text = ' '.join(words)
    return text

In [59]:
df3.head()

,id,name,tags
0,1,huawei,screen size camera ramgb battery mah operating...
1,2,duranta,this bicycle has a strong steel frame its brak...
2,19,cadillac,basic years miles corrosion years miles drivet...
3,20,aston martin,years unlimited miles basic years unlimited mi...
4,21,nike,you can buy the gym bag traveling bag nike onl...


In [60]:
df3['tags'] = df3['tags'].apply(textPorterStemmer)

In [61]:
df3.loc[1, 'tags']

'bicycl strong steel frame brake system strong allow maintain control situat seat adjust chang need front rear wheel fit mudguard rider get mud ride roaddurantaduranta super sport bicycl'

## Apply CounterVectorizer

In [62]:
cv = CountVectorizer(max_features=1200)

In [63]:
cv_vectors = cv.fit_transform(raw_documents=df3['tags']).toarray()

In [64]:
cv_vectors.shape

(153, 1200)

In [65]:
cv_vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [66]:
cv_vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [67]:
cv_similarity = cosine_similarity(X=cv_vectors)

In [68]:
cv_similarity.shape

(153, 153)

In [69]:
cv.get_feature_names_out()

array(['010', '013', '018', ..., 'youtub', 'zari', 'zigzag'], dtype=object)

In [70]:
cv_similarity[0]

array([1.        , 0.        , 0.        , 0.        , 0.        ,
       0.07715167, 0.        , 0.        , 0.        , 0.        ,
       0.06131393, 0.        , 0.        , 0.        , 0.        ,
       0.07412493, 0.14824986, 0.        , 0.09759001, 0.11952286,
       0.        , 0.21428571, 0.        , 0.        , 0.        ,
       0.        , 0.07142857, 0.        , 0.07412493, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.06131393, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.05976143, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

### Recommendar Function for PorterStemmer

In [76]:
def productRecommender2(productName):
    productName = productName.lower()
    product_index = df3[df3['name'] == productName].index[0]
    distances = sorted(list(enumerate(cv_similarity[product_index])), reverse=True, key=lambda x : x[1])
    for i in distances[1:16]:
        productID, productName = df3.loc[i[0], ['id','name']]
        print(f"Product ID:[{productID}] | Product Name: {productName}")
        # print(df3.loc[i[0], 'name')

In [72]:
productRecommender2('nike')

Product ID:[22] | Product Name: ladies backpack
Product ID:[164] | Product Name: laptop pouch bag
Product ID:[23] | Product Name: ladies backpack
Product ID:[100] | Product Name: baby gym rack and game pad combo
Product ID:[67] | Product Name: lg 32lk510b 32" hd led television
Product ID:[130] | Product Name: multi-function business travel wallet
Product ID:[97] | Product Name: led light
Product ID:[2] | Product Name: duranta
Product ID:[1] | Product Name: huawei
Product ID:[19] | Product Name: cadillac
Product ID:[20] | Product Name: aston martin
Product ID:[24] | Product Name: lakmé
Product ID:[25] | Product Name: lakmé 9 to 5
Product ID:[26] | Product Name: hijab
Product ID:[27] | Product Name: forever bicycle


In [77]:
productRecommender2("Swivel Chair")

Product ID:[177] | Product Name: hatil swivel chair
Product ID:[176] | Product Name: all swivel chair
Product ID:[71] | Product Name: gfc glamour ceiling fan
Product ID:[81] | Product Name: sennheiser ew135p-g4 wireless microphone
Product ID:[82] | Product Name: melody 3.5 octave stick harmoniam
Product ID:[136] | Product Name: silk readymade lehenga
Product ID:[164] | Product Name: laptop pouch bag
Product ID:[161] | Product Name: blender vis-sbl-018 rapid power
Product ID:[186] | Product Name: apple magic mouse
Product ID:[179] | Product Name: cpu liquid cooler
Product ID:[138] | Product Name: maithili silk saree
Product ID:[95] | Product Name: spot light
Product ID:[94] | Product Name: wled-rb3wb22 light
Product ID:[72] | Product Name: gfc monet ceiling fan
Product ID:[168] | Product Name: dual band gigabit 7 antennas router
